In [277]:
import pandas as pd
import math

In [278]:
def open_tabel(file):
    tabel_alternatif = pd.read_csv(file, index_col=0)
    
    return tabel_alternatif

In [279]:
def add_parameter(nilai, tabel_alternatif):
    tabel_alternatif["Parameter"] = nilai
    
    return tabel_alternatif

In [280]:
def get_jurusan(tabel_alternatif):
    pilihan_jurusan = []
    for column in tabel_alternatif:
        if column != "Parameter":
            pilihan_jurusan.append(column)
            
    return pilihan_jurusan

In [281]:
def deviasi(jurusan, jurusan_compare, index_name):
    nilai_deviasi = tabel_alternatif[jurusan][index_name] - tabel_alternatif[jurusan_compare][index_name]
    
    return nilai_deviasi

In [282]:
def fungsi_preferensi_gaussian(tabel_alternatif):
    fp_gaussian = {}
    fp_gaussian_val = {}

    for jurusan in pilihan_jurusan:
        for jurusan_compare in pilihan_jurusan:
            if jurusan == jurusan_compare:
                continue
            else:
                for index_name in tabel_alternatif.index:
                    if "(IPA)" in jurusan.split():
                        if index_name == "Matematika" or index_name == "Inggris" or index_name == "Indonesia" or index_name == "Fisika" or index_name == "Kimia" or index_name == "Biologi":

                            #menghitung deviasi
                            nilai_deviasi = deviasi(jurusan, jurusan_compare, index_name)

                            #menghitung fungsi preferensi gaussian
                            if nilai_deviasi > 0:
                                gaussian = 1 - math.exp(-1*(((nilai_deviasi)**2) / (2*((tabel_alternatif["Parameter"][index_name])**2))))
                            else:
                                gaussian = 0

                            deviasi_name = f"{jurusan}&{jurusan_compare}"
                            fp_gaussian_val[index_name] = gaussian

                    elif "(IPS)" in jurusan.split():
                        if index_name == "Ekonomi" or index_name == "Inggris" or index_name == "Indonesia" or index_name == "Geografi" or index_name == "Sosiologi" or index_name == "Sejarah":

                            #menghitung deviasi
                            nilai_deviasi = deviasi(jurusan, jurusan_compare, index_name)

                            #menghitung fungsi preferensi gaussian
                            if nilai_deviasi > 0:
                                gaussian = 1 - math.exp(-1*(((nilai_deviasi)**2) / (2*((tabel_alternatif["Parameter"][index_name])**2))))
                            else:
                                gaussian = 0

                            deviasi_name = f"{jurusan}&{jurusan_compare}"
                            fp_gaussian_val[index_name] = gaussian

                    try:
                        fp_gaussian[deviasi_name] = fp_gaussian_val
                    except NameError:
                        continue
            fp_gaussian_val = {}

    return fp_gaussian

In [283]:
def idx_pref_gaussian(fp_gaussian):
    index_preferensi_gaussian = {}
    weight = 1

    for compare, fgaussian in fp_gaussian.items():
        nilai_gaussian = 0
        for kriteria in fgaussian.keys():
            nilai_gaussian += fgaussian[kriteria]*weight
        nama_gaussian = compare
        index_preferensi_gaussian[nama_gaussian] = nilai_gaussian

    return index_preferensi_gaussian

In [284]:
def entering_flow(pilihan_jurusan, index_preferensi_gaussian):
    e_flow = {}

    for item in pilihan_jurusan:
        ef = 0
        for piece in index_preferensi_gaussian.keys():
            s = piece.split("&")

            if item in s and s.index(item) == 1:
                ef += index_preferensi_gaussian[piece] * (1/(len(pilihan_jurusan) - 1))

        e_flow[item] = ef

    return e_flow

In [285]:
def leaving_flow(pilihan_jurusan, index_preferensi_gaussian):
    l_flow = {}

    for item in pilihan_jurusan:
        lf = 0
        for piece in index_preferensi_gaussian.keys():
            s = piece.split("&")

            if item in s and s.index(item) == 0:
                lf += index_preferensi_gaussian[piece] * (1/(len(pilihan_jurusan) - 1))

        l_flow[item] = lf

    return l_flow

In [286]:
def net_flow(pilihan_jurusan, e_flow, l_flow):
    n_flow = {}

    for jurusan in pilihan_jurusan:
        n_flow[jurusan] = l_flow[jurusan] - e_flow[jurusan]

    return n_flow

In [287]:
def get_std(tabel_alternatif):
    #nyari simpangan baku
    std = tabel_alternatif.std(axis=1)

    return std

In [288]:
def get_mean(tabel_alternatif):
    #nyari rata2
    mean = tabel_alternatif.mean(axis=1)

    return mean

In [289]:
def get_params(mean, nilai, std):
    params = []
    for bjg in range(len(nilai_jams)):
        tot = (mean[bjg]-nilai_jams[bjg]) / std[bjg]
        params.append(tot)

    return params

In [290]:
tabel_alternatif = open_tabel("Tabel Alternatif.csv")

tabel_alternatif

,Administrasi Fiskal - UI (IPS),Administrasi Negara - UNPAD (IPS),Administrasi Niaga - UI (IPS),Administrasi Perpajakan - UB (IPS),Agribisnis - IPB (IPA),Agroekoteknologi - UB (IPA),Agronomi Dan Holtikultura - IPB (IPA),Agroteknologi - UNPAD (IPA),Akuntansi - UGM (IPS),Akutansi - UI (IPS),...,Teknologi Dan Manajemen Perikanan Budidaya - IPB (IPS),Teknologi Hasil Perikanan - UB (IPA),Teknologi Industri Hasil Perikanan - UNAIR (IPA),Teknologi Industri Pertanian - IPB (IPA),Teknologi Informasi - UGM (IPA),Teknologi Pangan - IPB (IPA),Televisi & Film - UNPAD (IPS),Tin - IPB (IPA),Tip - UGM (IPA),Tv & Film - UNPAD (IPS)
Matematika,84.000000,88.500000,85.465625,92.000000,95.000,81.852143,91.000000,82.500000,91.625000,44.693125,...,84.500000,82.000000,85.000000,85.410699,92.000000,92.000000,85.000000,90.00,87.000000,85.000000
Inggris,84.000000,80.500000,83.888667,86.000000,90.000,84.729722,91.000000,82.750000,90.875000,54.255467,...,85.500000,91.000000,84.000000,84.775000,90.000000,88.000000,85.500000,83.00,94.000000,85.000000
Indonesia,84.000000,89.500000,85.246032,89.000000,88.000,85.837917,89.000000,84.000000,89.875000,85.000000,...,86.500000,89.000000,86.000000,86.789540,93.000000,86.000000,86.500000,89.00,80.000000,84.000000
Fisika,82.000000,84.584430,84.493016,81.956615,91.000,81.739154,89.000000,80.500000,89.437576,87.000000,...,84.500000,80.000000,83.000000,85.810345,89.000000,92.000000,84.584430,88.00,86.000000,84.584430
Kimia,81.000000,84.592411,85.297674,82.397782,89.000,82.349446,90.000000,81.500000,88.716106,74.300000,...,84.000000,79.000000,81.000000,84.825000,89.000000,85.000000,84.592411,88.00,87.000000,84.592411
Biologi,83.000000,83.879942,84.182558,82.827536,90.000,83.706884,91.000000,83.250000,89.695798,83.000000,...,84.000000,83.000000,86.000000,86.391667,93.000000,86.000000,83.879942,89.00,86.000000,83.879942
Ekonomi,86.145714,89.000000,86.145714,91.242143,85.000,83.726429,83.000000,82.062045,92.250000,86.145714,...,84.422980,88.726429,85.211111,84.961490,89.837895,85.845960,86.000000,83.00,89.837895,86.000000
Geografi,85.512727,87.500000,85.512727,91.000000,81.316,84.627747,83.526667,81.926031,88.959803,85.512727,...,85.388333,84.808608,84.793706,83.645000,88.094322,87.250000,86.500000,87.25,88.094322,87.000000
Sosiologi,86.930877,85.500000,86.930877,91.000000,NaN,85.702500,NaN,81.331846,91.872475,86.930877,...,NaN,84.850000,85.695971,NaN,90.363636,NaN,85.500000,NaN,90.363636,85.000000
Sejarah,83.943883,88.000000,83.943883,93.000000,89.000,85.517917,82.676471,81.000669,90.581317,85.000000,...,85.958333,83.000000,84.000000,85.088235,91.000000,85.916667,86.500000,88.00,88.650538,85.000000


In [291]:
nilai_jams = [83.34, 82, 90, 85.2, 82, 83.4, 0, 0, 0, 0]

In [292]:
mean = get_mean(tabel_alternatif)
std = get_std(tabel_alternatif)
params = get_params(mean, nilai_jams, std)

params

[0.4542740371752141,
 0.9410931337655789,
 -1.1037905776184553,
 0.036439704737186565,
 0.9114268837945136,
 0.7903262703508075,
 32.341388290551855,
 35.59979369647639,
 29.164717473835555,
 29.360471236229703]

In [293]:
tabel_alternatif = add_parameter(params, tabel_alternatif)

tabel_alternatif

,Administrasi Fiskal - UI (IPS),Administrasi Negara - UNPAD (IPS),Administrasi Niaga - UI (IPS),Administrasi Perpajakan - UB (IPS),Agribisnis - IPB (IPA),Agroekoteknologi - UB (IPA),Agronomi Dan Holtikultura - IPB (IPA),Agroteknologi - UNPAD (IPA),Akuntansi - UGM (IPS),Akutansi - UI (IPS),...,Teknologi Hasil Perikanan - UB (IPA),Teknologi Industri Hasil Perikanan - UNAIR (IPA),Teknologi Industri Pertanian - IPB (IPA),Teknologi Informasi - UGM (IPA),Teknologi Pangan - IPB (IPA),Televisi & Film - UNPAD (IPS),Tin - IPB (IPA),Tip - UGM (IPA),Tv & Film - UNPAD (IPS),Parameter
Matematika,84.000000,88.500000,85.465625,92.000000,95.000,81.852143,91.000000,82.500000,91.625000,44.693125,...,82.000000,85.000000,85.410699,92.000000,92.000000,85.000000,90.00,87.000000,85.000000,0.454274
Inggris,84.000000,80.500000,83.888667,86.000000,90.000,84.729722,91.000000,82.750000,90.875000,54.255467,...,91.000000,84.000000,84.775000,90.000000,88.000000,85.500000,83.00,94.000000,85.000000,0.941093
Indonesia,84.000000,89.500000,85.246032,89.000000,88.000,85.837917,89.000000,84.000000,89.875000,85.000000,...,89.000000,86.000000,86.789540,93.000000,86.000000,86.500000,89.00,80.000000,84.000000,-1.103791
Fisika,82.000000,84.584430,84.493016,81.956615,91.000,81.739154,89.000000,80.500000,89.437576,87.000000,...,80.000000,83.000000,85.810345,89.000000,92.000000,84.584430,88.00,86.000000,84.584430,0.036440
Kimia,81.000000,84.592411,85.297674,82.397782,89.000,82.349446,90.000000,81.500000,88.716106,74.300000,...,79.000000,81.000000,84.825000,89.000000,85.000000,84.592411,88.00,87.000000,84.592411,0.911427
Biologi,83.000000,83.879942,84.182558,82.827536,90.000,83.706884,91.000000,83.250000,89.695798,83.000000,...,83.000000,86.000000,86.391667,93.000000,86.000000,83.879942,89.00,86.000000,83.879942,0.790326
Ekonomi,86.145714,89.000000,86.145714,91.242143,85.000,83.726429,83.000000,82.062045,92.250000,86.145714,...,88.726429,85.211111,84.961490,89.837895,85.845960,86.000000,83.00,89.837895,86.000000,32.341388
Geografi,85.512727,87.500000,85.512727,91.000000,81.316,84.627747,83.526667,81.926031,88.959803,85.512727,...,84.808608,84.793706,83.645000,88.094322,87.250000,86.500000,87.25,88.094322,87.000000,35.599794
Sosiologi,86.930877,85.500000,86.930877,91.000000,NaN,85.702500,NaN,81.331846,91.872475,86.930877,...,84.850000,85.695971,NaN,90.363636,NaN,85.500000,NaN,90.363636,85.000000,29.164717
Sejarah,83.943883,88.000000,83.943883,93.000000,89.000,85.517917,82.676471,81.000669,90.581317,85.000000,...,83.000000,84.000000,85.088235,91.000000,85.916667,86.500000,88.00,88.650538,85.000000,29.360471


In [294]:
# pilihan_jurusan = ["Stei - ITB (IPA)", "Administrasi Fiskal - UI (IPS)", "Akuntansi - UGM (IPS)"]

In [295]:
pilihan_jurusan = get_jurusan(tabel_alternatif)

pilihan_jurusan

['Administrasi Fiskal - UI (IPS)',
 'Administrasi Negara - UNPAD (IPS)',
 'Administrasi Niaga - UI (IPS)',
 'Administrasi Perpajakan - UB (IPS)',
 'Agribisnis - IPB (IPA)',
 'Agroekoteknologi - UB (IPA)',
 'Agronomi Dan Holtikultura - IPB (IPA)',
 'Agroteknologi - UNPAD (IPA)',
 'Akuntansi - UGM (IPS)',
 'Akutansi - UI (IPS)',
 'Antropologi - UNAIR (IPA)',
 'Antropologi Sosial - UB (IPS)',
 'Apoteker - UNAIR (IPA)',
 'Arkeologi - UI (IPA)',
 'Arsitektur - ITB (IPA)',
 'Arsitektur Lansekap - IPB (IPA)',
 'Astronomi - ITB (IPA)',
 'Biokimia - IPB (IPA)',
 'Biologi - ITS (IPA)',
 'Budidaya Perairan - UNAIR (IPA)',
 'Desain Interior - ITS (IPS)',
 'Ekonomi Dan Studi Pembangunan - IPB (IPS)',
 'Ekonomi Islam - UB (IPS)',
 'Ekonomi Islam  - UNPAD (IPS)',
 'Ekonomi Pembangunan - IPB (IPS)',
 'Elektro - ITB (IPA)',
 'Fakultas Kedokteran Gigi - UNAIR (IPA)',
 'Fakultas Kedokteran Hewan - IPB (IPA)',
 'Fakultas Teknik Industri - ITB (IPA)',
 'Farmasi - UI (IPA)',
 'Fisika - IPB (IPA)',
 'Fiskal 

In [296]:
fp_gaussian = fungsi_preferensi_gaussian(tabel_alternatif)

fp_gaussian

{'Administrasi Fiskal - UI (IPS)&Administrasi Negara - UNPAD (IPS)': {'Inggris': 0.006973259931829623,
  'Indonesia': 0,
  'Ekonomi': 0,
  'Geografi': 0,
  'Sosiologi': 0,
  'Sejarah': 0},
 'Administrasi Fiskal - UI (IPS)&Administrasi Niaga - UI (IPS)': {'Inggris': 0,
  'Indonesia': 0,
  'Ekonomi': 0,
  'Geografi': 0,
  'Sosiologi': 0,
  'Sejarah': 0},
 'Administrasi Fiskal - UI (IPS)&Administrasi Perpajakan - UB (IPS)': {'Inggris': 0,
  'Indonesia': 0,
  'Ekonomi': 0.0006272910535218834,
  'Geografi': 0.006924501973726294,
  'Sosiologi': 0,
  'Sejarah': 0},
 'Administrasi Fiskal - UI (IPS)&Agribisnis - IPB (IPA)': {'Inggris': 0,
  'Indonesia': 0,
  'Ekonomi': 0.002793956253211771,
  'Geografi': 0.0003089405243722476,
  'Sosiologi': 0.0008865950105871878,
  'Sejarah': 0},
 'Administrasi Fiskal - UI (IPS)&Agroekoteknologi - UB (IPA)': {'Inggris': 0,
  'Indonesia': 0,
  'Ekonomi': 0.004719157410527641,
  'Geografi': 0.0015549705195572194,
  'Sosiologi': 0,
  'Sejarah': 0.0009312747223066

In [297]:
index_preferensi_gaussian = idx_pref_gaussian(fp_gaussian)

idx_pref_gaussian(fp_gaussian)

{'Administrasi Fiskal - UI (IPS)&Administrasi Negara - UNPAD (IPS)': 0.006973259931829623,
 'Administrasi Fiskal - UI (IPS)&Administrasi Niaga - UI (IPS)': 0,
 'Administrasi Fiskal - UI (IPS)&Administrasi Perpajakan - UB (IPS)': 0.007551793027248177,
 'Administrasi Fiskal - UI (IPS)&Agribisnis - IPB (IPA)': 0.003989491788171207,
 'Administrasi Fiskal - UI (IPS)&Agroekoteknologi - UB (IPA)': 0.007205402652391557,
 'Administrasi Fiskal - UI (IPS)&Agronomi Dan Holtikultura - IPB (IPA)': 0.6223668022982876,
 'Administrasi Fiskal - UI (IPS)&Agroteknologi - UNPAD (IPA)': 0,
 'Administrasi Fiskal - UI (IPS)&Akuntansi - UGM (IPS)': 1.0,
 'Administrasi Fiskal - UI (IPS)&Akutansi - UI (IPS)': 0.0015628827565141723,
 'Administrasi Fiskal - UI (IPS)&Antropologi - UNAIR (IPA)': 2.0627004969690423,
 'Administrasi Fiskal - UI (IPS)&Antropologi Sosial - UB (IPS)': 0.0014383564469792054,
 'Administrasi Fiskal - UI (IPS)&Apoteker - UNAIR (IPA)': 0.00022855977904567837,
 'Administrasi Fiskal - UI (IPS)&A

In [298]:
e_flow = entering_flow(pilihan_jurusan, index_preferensi_gaussian)

e_flow

{'Administrasi Fiskal - UI (IPS)': 3.401987893391837,
 'Administrasi Negara - UNPAD (IPS)': 2.362217327706882,
 'Administrasi Niaga - UI (IPS)': 2.4495389765186273,
 'Administrasi Perpajakan - UB (IPS)': 2.1344024660414203,
 'Agribisnis - IPB (IPA)': 0.5531144393393822,
 'Agroekoteknologi - UB (IPA)': 2.8913869991902352,
 'Agronomi Dan Holtikultura - IPB (IPA)': 0.7387306794610611,
 'Agroteknologi - UNPAD (IPA)': 3.2867666960218513,
 'Akuntansi - UGM (IPS)': 0.8012353417983986,
 'Akutansi - UI (IPS)': 3.257722949327763,
 'Antropologi - UNAIR (IPA)': 2.6175950485009065,
 'Antropologi Sosial - UB (IPS)': 3.975978046740485,
 'Apoteker - UNAIR (IPA)': 1.6796535834520403,
 'Arkeologi - UI (IPA)': 2.1768593949019057,
 'Arsitektur - ITB (IPA)': 1.2563279737870654,
 'Arsitektur Lansekap - IPB (IPA)': 1.8059276137530793,
 'Astronomi - ITB (IPA)': 2.195074313191133,
 'Biokimia - IPB (IPA)': 1.8487618807748214,
 'Biologi - ITS (IPA)': 1.8913450674912236,
 'Budidaya Perairan - UNAIR (IPA)': 3.0280

In [299]:
l_flow = leaving_flow(pilihan_jurusan, index_preferensi_gaussian)

l_flow

{'Administrasi Fiskal - UI (IPS)': 0.2534029754555872,
 'Administrasi Negara - UNPAD (IPS)': 0.7870002220637946,
 'Administrasi Niaga - UI (IPS)': 0.34064562468893367,
 'Administrasi Perpajakan - UB (IPS)': 1.143560450644376,
 'Agribisnis - IPB (IPA)': 4.895909144960034,
 'Agroekoteknologi - UB (IPA)': 0.8585692694029436,
 'Agronomi Dan Holtikultura - IPB (IPA)': 5.0309964608215365,
 'Agroteknologi - UNPAD (IPA)': 0.5248639342782487,
 'Akuntansi - UGM (IPS)': 1.6941141452979507,
 'Akutansi - UI (IPS)': 0.17112900113192678,
 'Antropologi - UNAIR (IPA)': 2.002278588577039,
 'Antropologi Sosial - UB (IPS)': 0.04663719396503019,
 'Apoteker - UNAIR (IPA)': 2.9089030385576575,
 'Arkeologi - UI (IPA)': 1.8013260767761898,
 'Arsitektur - ITB (IPA)': 4.02979137007678,
 'Arsitektur Lansekap - IPB (IPA)': 3.0251147116043593,
 'Astronomi - ITB (IPA)': 2.28784462519526,
 'Biokimia - IPB (IPA)': 2.552114974488842,
 'Biologi - ITS (IPA)': 2.699986080480475,
 'Budidaya Perairan - UNAIR (IPA)': 0.90949

In [300]:
n_flow = net_flow(pilihan_jurusan, e_flow, l_flow)

n_flow

{'Administrasi Fiskal - UI (IPS)': -3.1485849179362497,
 'Administrasi Negara - UNPAD (IPS)': -1.5752171056430873,
 'Administrasi Niaga - UI (IPS)': -2.1088933518296935,
 'Administrasi Perpajakan - UB (IPS)': -0.9908420153970443,
 'Agribisnis - IPB (IPA)': 4.342794705620651,
 'Agroekoteknologi - UB (IPA)': -2.0328177297872916,
 'Agronomi Dan Holtikultura - IPB (IPA)': 4.292265781360475,
 'Agroteknologi - UNPAD (IPA)': -2.7619027617436025,
 'Akuntansi - UGM (IPS)': 0.8928788034995521,
 'Akutansi - UI (IPS)': -3.086593948195836,
 'Antropologi - UNAIR (IPA)': -0.6153164599238674,
 'Antropologi Sosial - UB (IPS)': -3.9293408527754545,
 'Apoteker - UNAIR (IPA)': 1.2292494551056172,
 'Arkeologi - UI (IPA)': -0.3755333181257159,
 'Arsitektur - ITB (IPA)': 2.773463396289715,
 'Arsitektur Lansekap - IPB (IPA)': 1.21918709785128,
 'Astronomi - ITB (IPA)': 0.09277031200412722,
 'Biokimia - IPB (IPA)': 0.7033530937140207,
 'Biologi - ITS (IPA)': 0.8086410129892516,
 'Budidaya Perairan - UNAIR (IPA

In [301]:
hasil_outrank = sorted(n_flow.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)

tab = dict()

rank = list()
jurusan = list()
net = list()

for i in range(len(hasil_outrank)):
    rank.append(i+1)
    jurusan.append(hasil_outrank[i][0])
    net.append(hasil_outrank[i][1])
    
tab["Jurusan"] = jurusan
tab["Net Flow"] = net

tabel_hasil = pd.DataFrame(tab, index=rank)

tabel_hasil

,Jurusan,Net Flow
1,S-1 Teknik Informatika - ITS (IPA),5.649260
2,Fmipa Fisika - UB (IPA),5.466787
3,Pend. Apoteker - UNAIR (IPA),5.048155
4,Fakultas Kedokteran Hewan - IPB (IPA),4.832227
5,Teknologi Informasi - UGM (IPA),4.830755
...,...,...
158,Manajemen Komunikasi - UNPAD (IPS),-3.480135
159,Hukum - UB (IPS),-3.583219
160,Ilmu Politik - UB (IPS),-3.600458
161,Teknik Lingkungan - UB (IPA),-3.606018
